In [1]:
import praw
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import json
import requests
import os.path

In [2]:
def api_request(subreddit, startDate, endDate):
    size = 100
    host = "https://api.pushshift.io/reddit/search/submission"
    try:
        posts = requests.get(host,
                headers={'User-Agent': 'TextMining'},
                params={'subreddit': subreddit,
                     'before': endDate,
                     'after': startDate,
                     'metadata': False,
                     'limit': 100})
        data = posts.json()['data']
        print(startDate, len(data), len(posts.json()['data']))
    except JSONDecodeError:
        print('Bad gateway error. Retrying…')
    return posts

In [3]:
def read_text(subreddit):
    if (os.path.isfile('r_' + subreddit + '.txt') == True):
        submissions = {}

    with open('r_' + subreddit + '.txt', 'r') as f:
        submissions['r/' + subreddit] = json.loads(f.read())
        return submissions

def manage_posts(posts, subreddit):
    if (os.path.isfile('r_' + subreddit + '.txt') == False):
        with open('r_' + subreddit + '.txt', 'a') as f:
            json.dump(posts, f)

    return read_text(subreddit);

In [4]:
# Defining dates according to Unix time.

time_periods = [[1609459200, 1612137600], # 01-01-21 - 01-02-21
              [1612224000, 1614556800], # 02-02-21 - 01-03-21
              [1614643200, 1617235200], # 02-03-21 - 01-04-21
              [1617321600, 1619827200], # 02-04-21 - 01-05-21
              [1619913600, 1622505600], # 02-05-21 - 01-06-21
              [1622592000, 1625097600], # 02-06-21 - 01-07-21
              [1625184000, 1627776000], # 02-07-21 - 01-08-21
              [1627862400, 1630454400], # 02-08-21 - 01-09-21
              [1630540800, 1633046400], # 02-09-21 - 01-10-21
              [1633132800, 1635724800], # 02-10-21 - 01-11-21
              [1635811200, 1638316800], # 02-11-21 - 01-12-21
              [1638403200, 1640908800]] # 02-12-21 - 31-12-21

posts_stress = []
posts_depression = []
posts_anxiety = []
posts_bipolar = []
posts_ptsd = []
posts_eatingdisorder = []
posts_schizophrenia = []

def test(posts):
    all_posts = []
    for single_post in range(len(posts.json()['data'])):
        all_posts.append(posts.json()['data'][single_post])
    return all_posts

for period in time_periods:
    stress = api_request("stress", period[0], period[1])
    depression = api_request("depression", period[0], period[1])
    anxiety = api_request("anxiety", period[0], period[1])
    bipolar = api_request("bipolar", period[0], period[1])
    ptsd = api_request("ptsd", period[0], period[1])
    eatingdisorder = api_request("eatingdisorder", period[0], period[1])
    schizophrenia = api_request("schizophrenia", period[0], period[1])
    
    posts_stress = test(stress)
    posts_depression = test(depression)
    posts_anxiety = test(anxiety)
    posts_bipolar = test(bipolar)
    posts_ptsd = test(ptsd)
    posts_eatingdisorder = test(eatingdisorder)
    posts_schizophrenia = test(schizophrenia)

stress_json = manage_posts(posts_stress, "stress")
depression_json = manage_posts(posts_depression, "depression")
anxiety_json = manage_posts(posts_anxiety, "anxiety")
bipolar_json = manage_posts(posts_bipolar, "bipolar")
ptsd_json = manage_posts(posts_ptsd, "ptsd")
eatingdisorder_json = manage_posts(posts_eatingdisorder, "eatingdisorder")
schizophrenia_json = manage_posts(posts_schizophrenia, "schizophrenia")

1609459200 100 100
1609459200 100 100
1609459200 100 100
1609459200 99 99
1609459200 100 100
1609459200 57 57
1609459200 99 99
1612224000 85 85
1612224000 100 100
1612224000 100 100
1612224000 100 100
1612224000 100 100
1612224000 41 41
1612224000 100 100
1614643200 90 90
1614643200 100 100
1614643200 100 100
1614643200 99 99
1614643200 100 100
1614643200 34 34
1614643200 100 100
1617321600 100 100
1617321600 100 100
1617321600 100 100
1617321600 100 100
1617321600 100 100
1617321600 73 73
1617321600 99 99
1619913600 100 100
1619913600 100 100
1619913600 100 100
1619913600 100 100
1619913600 100 100
1619913600 73 73
1619913600 100 100
1622592000 100 100
1622592000 100 100
1622592000 100 100
1622592000 99 99
1622592000 100 100
1622592000 61 61
1622592000 100 100
1625184000 100 100
1625184000 100 100
1625184000 100 100
1625184000 100 100
1625184000 100 100
1625184000 62 62
1625184000 100 100
1627862400 100 100
1627862400 100 100
1627862400 100 100
1627862400 100 100
1627862400 99 99
1627

In [5]:
def get_dataframe(subreddit_json):
    for subreddit, posts in subreddit_json.items():
        subreddit_df = pd.DataFrame(posts, columns=['id', 'created_utc',
                       'author', 'title', 'selftext', 'subreddit_subscribers'])
        pd.options.display.max_colwidth = 1000
        subreddit_df.columns = ['ID', 'Date created', 'Author', 'Title', 'Selftext', 'Subscribers']

        # The way fetching data works, it is very possible that we also have some entries that are outside the defined period (Jan–Jun 2018, 2019, 2020). We are going to remove them after converting the dates from unix time to pandas' own timestamps.
        subreddit_df['Date created'] = pd.to_datetime(subreddit_df['Date created'], unit='s')

    return subreddit_df

In [6]:
stress_df = get_dataframe(stress_json)
depression_df = get_dataframe(depression_json)
anxiety_df = get_dataframe(anxiety_json)
bipolar_df = get_dataframe(bipolar_json)
ptsd_df = get_dataframe(ptsd_json)
eatingdisorder_df = get_dataframe(eatingdisorder_json)
schizophrenia_df = get_dataframe(schizophrenia_json)

%store stress_df
%store depression_df
%store anxiety_df
%store bipolar_df
%store ptsd_df
%store eatingdisorder_df
%store schizophrenia_df

stress_df.tail()

Stored 'stress_df' (DataFrame)
Stored 'depression_df' (DataFrame)
Stored 'anxiety_df' (DataFrame)
Stored 'bipolar_df' (DataFrame)
Stored 'ptsd_df' (DataFrame)
Stored 'eatingdisorder_df' (DataFrame)
Stored 'schizophrenia_df' (DataFrame)


,ID,Date created,Author,Title,Selftext,Subscribers
95,rqmfko,2021-12-28 18:36:56,Effective_Plenty2040,(Playlist) Let's get lost in our own world and leave our worries behind. Let's chill and remove the stress from our minds shall we?,https://youtu.be/PkSHnmpRGDg,12603
96,rqstu5,2021-12-28 23:26:05,amandasbitch,Ugh!,"So I was diagnosed with congestive heart failure a few months back. On top of the meds I now have to take for my weak heart, Drs stress that I avoid stress as it will raise my pressure and heart rate. I had to leave my job due to this condition, social security is taking their sweet time and I am now broke and still have bills and my kids to take care of. \n Does anyone have some natural ways I can try and manage the stress of all of this without doing more/worse damage to my heart. I am literally sleeping maybe 2 hours a night and I have tried meditation, ASMR etc etc. Nothing is helping at all. I need to de stress! \n Thank you for listening!",12604
97,rqxbgz,2021-12-29 02:57:11,itsjay88,Nightmares While Awake,"Influence, manipulation, hopelessness.\n\nDown the rabbit hole we all go. \n\nFor inaction is the same as giving permission. The absence of resistance is willingness to be played with like a toy.",12604
98,rr3dfq,2021-12-29 08:33:39,T-IKEICH,"Caring | Piano Music for studying, Focus Work, Relaxing, Stress Relief, Spa, Calm Mind Vibes",“The simple act of caring is heroic.” — Edward Albert \n\n[https://youtu.be/yIXpkSmYIlI](https://youtu.be/yIXpkSmYIlI),12603
99,rrmwz4,2021-12-30 00:05:18,SeriousPuppet,I wish I could talk to someone about my stress,"I have tried a therapist and a life coach. Didn't really help much. Was nice to chat with them sure, but I feel it would be just as good chatting with a random person.",12604


In [7]:
stress_df['Selftext'][0]

'**I have a 57.75 in Math right now. I have less than 2 weeks to bring up my math grade from an F to a D (3 points) before I need to go to summer school and re-take it next year, and maybe not graduate. I am a Sophomore in high-school. Math has never been something I am good at. I have 2 more tests, including one tomorrow. My parents are yelling at me and stressing me out and making my self harming issue worse, and they wont get me a tutor. Ive been having dizzy spells and fainting spells because im so stressed that I forget to take care of myself and life is shit. WHAT do I do???**'

In [8]:
anxiety_df['Selftext'][0]

"Sometimes we just need a hug, especially with everything going on right now. So, for anyone that has nobody to talk to, or anyone healing from past things they don't talk about, or for anyone that just needs a hug. This is virtual hug from me to you. 🤗"

In [9]:
bipolar_df['Selftext'][0]

"What if we all were wearing HUD heads up display and we could see text bubbles when people spoke and conversations were always recorded. Imagine a world where everybody was a surveillance device with HUDS monitoring audio communications more advanced then Google glasses. Sorry if I'm triggering those with paranoia. These are some fleeting thoughts I get when listening to binaural beats. I feel like I'm one step removed from some important truth. My mind is becoming free in a healthy way. Earth wind water fire air elements my brain is trying to become. I'm entering a void devoid of limits where am I headed to. My mind feels in place yet far into space..."

In [10]:
ptsd_df['Selftext'][0]

'my boyfriend had been sexually assaulting me and raped me, but i didn’t realize what was really happening. when i did, i talked it through w/ him. he said sorry, won’t do it again &amp; cried. so we’re moving on. but i feel it hurt me a lot, emotionally. i feel like my body isn’t mine, it’s dirty, i want to take it off. i cry often now, thinking abt the rape all the time. i’m so hurt. what do i do? i still love him'

In [11]:
eatingdisorder_df['Selftext'][19]

'Today I went out with my aunt and we ate kraftwich for lunch. We both order our own portion of a kraftwich, mushroom soup and a drink (mine latte, hers lemon tea). Usually I would NEVER finish my food whilst she does. Today was the other way round, I finished my kraftwich and mushroom and left out my latte because it was TOO COLD!!!! Whereas she didnt finish her kraftwich but finished her mushroom soup and drink. \n\nAfter watching her not finishing her food, I felt really bad for myself as I have actually swallowed a huge sandwich and soup whilst she swallowed her soup and drink. \n\nP. S. \nJust a sharing...'

In [12]:
schizophrenia_df['Selftext'][0]

'So what is a mental breakdown?'

In [13]:
schizophrenia_df['Selftext'][1]

'Thank you all for being there and listening 🥰.\n\nYou all rock and I hope you have a wonderful day.'